In [1]:
import os

In [2]:
pwd

'd:\\cnn-example\\cnn-chickendisease-example\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'd:\\cnn-example\\cnn-chickendisease-example'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [12]:
class ConfigurationManager:
    "configuration manager class"
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    


    def get_training_config(self) -> TrainingConfig:
        '''
        In this first section, you're gathering all the necessary pieces of configuration. 
        Think of it like getting all your ingredients ready before cooking:
        the training settings, the base model settings, and the general parameters are all pulled from your YAML files and stored in easy-to-use variables.
        
        '''
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "chickenfecalimages")
        create_directories([
            Path(training.root_dir)
        ])
        '''
        Here, you're setting up your workspace. You're creating the path to your training data 
        (combining the unzip directory path with the "chickenfecalimages" folder name) 
        and ensuring your training directory exists
        '''

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config
    
    '''
    This creates a comprehensive training configuration package. Let's understand each parameter:

     root_dir: The main directory where training outputs will be saved
     trained_model_path: Where your final trained model will be saved
     updated_base_model_path: Where your modified base model is located (probably from a previous preparation step)
     training_data: Path to your dataset
     params_epochs: How many times to train over the entire dataset
     params_batch_size: How many images to process at once
     params_is_augmentation: Whether to use data augmentation (like flipping or rotating images)
     params_image_size: The size your images need to be for training
    '''

In [13]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [14]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        "load the updated base model as we will finetune this"
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
    '''
    the above 2 functions loads your pre-prepared base model, 
    which will be fine-tuned during training. 
    Think of it like getting a skilled artist (the pre-trained model)
    and then teaching them specifically about chicken fecal images.
    
    '''

    def train_valid_generator(self):
        '''
        this method prepares your data 
        
        datagenerator_kwargs your image data (converting pixel values from 0-255 to 0-1) 
        and sets aside 20% of your data for validation. 
        It's like preparing your test materials before teaching begins.
        
        '''

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
        '''
        The below code : 
        When augmentation is enabled, it creates variations of your training images by 
        rotating, flipping, and zooming. This helps your model learn to recognize images 
        from different angles and positions - 
        like teaching someone to recognize objects from multiple viewpoints.
        
        '''

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    
    def train(self):
        '''
        This is where the learning happens. 
        The model processes batches of images repeatedly (epochs), adjusting its understanding with each pass. 
        The steps_per_epoch calculation ensures we use all our training data efficiently.
        
        '''
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            # This preserves your trained model so you can use it later for predictions.
            path=self.config.trained_model_path,
            model=self.model
        )

In [15]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

# convert this as modular coding,update in config_entity.py first

[2025-01-02 11:17:10,844: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-02 11:17:10,847: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-02 11:17:10,850: INFO: common: created directory at: artifacts]
[2025-01-02 11:17:10,852: INFO: common: created directory at: artifacts\training]
Found 10 images belonging to 2 classes.
Found 42 images belonging to 2 classes.
2/2 [==============================] - 5s 2s/step - loss: 0.6037 - accuracy: 0.7308
